# 새로 배운 kfold와 OOB앙상블 등을 사용해서 결과를 뽑아보겠습니다.

In [17]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')
import os, sys

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import KFold

In [2]:
TRAIN_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\train\train.csv'
TEST_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\test'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\sample_submission.csv'

In [62]:
train = ld.load_dtypes(TRAIN_PATH)
train.info()
submission = pd.read_csv(SUBMISSION_PATH)

C:\Users\Wyatt\wyatt37\Data\solarpanel\train\train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     52560 non-null  int16  
 1   Hour    52560 non-null  int8   
 2   Minute  52560 non-null  int8   
 3   DHI     52560 non-null  int16  
 4   DNI     52560 non-null  int16  
 5   WS      52560 non-null  float32
 6   RH      52560 non-null  float32
 7   T       52560 non-null  int8   
 8   TARGET  52560 non-null  float32
dtypes: float32(3), int16(3), int8(3)
memory usage: 1.1 MB


In [63]:
# hour와 minute을 합쳐주는 함수입니다.
def sum_hour_minute(train):
    # 0과 30으로 되어 있는 minute을 0과 0.5로 바꿉니다.
    train.Minute = round(train.Minute * 0.017, 1)
    # hour에 minute을 더해줍니다.
    train.Hour = train.Hour + train.Minute
    
    return train

In [64]:
# 최종적으로 파생변수 만드는 함수
def make_features(train):
    print("original_shape: ", train.shape)
    # hour와 minute 합쳐줍니다.
    train = sum_hour_minute(train)
    
    #-------------------- 파생변수 생성하면 이 아래에 추가

    #----------------------------------------------------
    train.drop(['Day', 'Minute'], axis=1, inplace=True)  
    
    print("feature_creationed: ", train.shape)
    
    return train

In [65]:
train = make_features(train)
train

original_shape:  (52560, 9)
feature_creationed:  (52560, 7)


,Hour,DHI,DNI,WS,RH,T,TARGET
0,0.0,0,0,1.5,69.080002,-12,0.0
1,0.5,0,0,1.5,69.059998,-12,0.0
2,1.0,0,0,1.6,71.779999,-12,0.0
3,1.5,0,0,1.6,71.750000,-12,0.0
4,2.0,0,0,1.6,75.199997,-12,0.0
...,...,...,...,...,...,...,...
52555,21.5,0,0,2.4,70.699997,-4,0.0
52556,22.0,0,0,2.4,66.790001,-4,0.0
52557,22.5,0,0,2.2,66.779999,-4,0.0
52558,23.0,0,0,2.1,67.720001,-4,0.0


# shift-window

하루씩 줄여가며 가보겠습니다. 테스트 데이터를 덜 쓴다는 단점이 있는데, 이것이 어떻게 작용할지 살펴봐야겠습니다.

In [66]:
shifted_columns = train.columns[1:]
lags = [48, 96, 144, 192, 240] # 뒤에서부터 하나씩 줄인다.

In [67]:
def shift_columns(data, shifted_columns, lags):
    
    temp = data.copy()
    for lag in lags:
        for col in shifted_columns:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            
    return temp

In [68]:
train = shift_columns(train, shifted_columns, lags)
train.shape

(52560, 37)

In [69]:
def make_targets(train):

    # 타겟값을 미리 만들어줘야 합니다.
    train['TARGET_1'] = train.TARGET.shift(-48)
    train['TARGET_2'] = train.TARGET_1.shift(-48)

    train.dropna(inplace=True)
    print(train.shape)
    
    return train

In [70]:
train = make_targets(train)
train

(52224, 39)


,Hour,DHI,DNI,WS,RH,T,TARGET,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,TARGET_lag_48,DHI_lag_96,DNI_lag_96,WS_lag_96,RH_lag_96,T_lag_96,TARGET_lag_96,DHI_lag_144,DNI_lag_144,WS_lag_144,RH_lag_144,T_lag_144,TARGET_lag_144,DHI_lag_192,DNI_lag_192,WS_lag_192,RH_lag_192,T_lag_192,TARGET_lag_192,DHI_lag_240,DNI_lag_240,WS_lag_240,RH_lag_240,T_lag_240,TARGET_lag_240,TARGET_1,TARGET_2
240,0.0,0,0,1.5,77.410004,1,0.0,0.0,0.0,2.1,72.099998,-5.0,0.0,0.0,0.0,1.8,77.760002,-14.0,0.0,0.0,0.0,2.2,73.800003,-8.0,0.0,0.0,0.0,1.6,90.660004,-10.0,0.0,0.0,0.0,1.5,69.080002,-12.0,0.0,0.0,0.0
241,0.5,0,0,1.7,77.419998,1,0.0,0.0,0.0,2.0,72.099998,-5.0,0.0,0.0,0.0,2.0,77.769997,-13.0,0.0,0.0,0.0,2.1,68.199997,-8.0,0.0,0.0,0.0,1.6,90.680000,-10.0,0.0,0.0,0.0,1.5,69.059998,-12.0,0.0,0.0,0.0
242,1.0,0,0,1.9,76.769997,1,0.0,0.0,0.0,2.0,72.139999,-5.0,0.0,0.0,0.0,2.2,77.690002,-13.0,0.0,0.0,0.0,2.1,69.059998,-8.0,0.0,0.0,0.0,1.6,88.110001,-11.0,0.0,0.0,0.0,1.6,71.779999,-12.0,0.0,0.0,0.0
243,1.5,0,0,2.0,76.769997,1,0.0,0.0,0.0,2.0,72.110001,-5.0,0.0,0.0,0.0,2.2,71.570000,-13.0,0.0,0.0,0.0,2.1,69.040001,-8.0,0.0,0.0,0.0,1.6,88.110001,-11.0,0.0,0.0,0.0,1.6,71.750000,-12.0,0.0,0.0,0.0
244,2.0,0,0,2.2,76.389999,1,0.0,0.0,0.0,2.0,72.940002,-5.0,0.0,0.0,0.0,2.2,72.620003,-13.0,0.0,0.0,0.0,2.2,69.379997,-8.0,0.0,0.0,0.0,1.6,90.849998,-11.0,0.0,0.0,0.0,1.6,75.199997,-12.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,21.5,0,0,3.5,55.970001,-1,0.0,0.0,0.0,3.3,61.599998,-1.0,0.0,0.0,0.0,2.7,57.009998,-1.0,0.0,0.0,0.0,1.0,84.849998,-8.0,0.0,0.0,0.0,1.7,79.269997,-4.0,0.0,0.0,0.0,1.6,48.889999,-3.0,0.0,0.0,0.0
52460,22.0,0,0,3.9,54.230000,-2,0.0,0.0,0.0,3.3,58.410000,-1.0,0.0,0.0,0.0,2.7,56.820000,-1.0,0.0,0.0,0.0,1.2,82.260002,-8.0,0.0,0.0,0.0,1.7,76.800003,-4.0,0.0,0.0,0.0,1.5,51.959999,-3.0,0.0,0.0,0.0
52461,22.5,0,0,4.1,54.209999,-2,0.0,0.0,0.0,3.3,58.380001,-1.0,0.0,0.0,0.0,2.6,56.820000,-1.0,0.0,0.0,0.0,1.3,82.250000,-8.0,0.0,0.0,0.0,1.6,76.800003,-4.0,0.0,0.0,0.0,1.4,51.950001,-3.0,0.0,0.0,0.0
52462,23.0,0,0,4.3,56.459999,-2,0.0,0.0,0.0,3.4,55.939999,-1.0,0.0,0.0,0.0,2.6,56.900002,-1.0,0.0,0.0,0.0,1.5,81.139999,-8.0,0.0,0.0,0.0,1.6,80.500000,-5.0,0.0,0.0,0.0,1.3,56.099998,-3.0,0.0,0.0,0.0


In [71]:
X = train[train.columns[:-2]]
y_1 = train['TARGET_1']
y_2 = train['TARGET_2']

In [72]:
class HiddenPrints: # print를 막아주는 클래스 입니다.
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
with HiddenPrints():
    df_test = []

    for i in range(81):
        file_path = TEST_PATH + '/' + str(i) + '.csv'
        temp = pd.read_csv(file_path)
        temp = make_features(temp)
        temp = shift_columns(temp, shifted_columns, lags).dropna()[-48:]
        df_test.append(temp)

    X_test = pd.concat(df_test)
print(X_test.shape)

(3888, 37)


In [73]:
X_test

,Hour,DHI,DNI,WS,RH,T,TARGET,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,TARGET_lag_48,DHI_lag_96,DNI_lag_96,WS_lag_96,RH_lag_96,T_lag_96,TARGET_lag_96,DHI_lag_144,DNI_lag_144,WS_lag_144,RH_lag_144,T_lag_144,TARGET_lag_144,DHI_lag_192,DNI_lag_192,WS_lag_192,RH_lag_192,T_lag_192,TARGET_lag_192,DHI_lag_240,DNI_lag_240,WS_lag_240,RH_lag_240,T_lag_240,TARGET_lag_240
288,0.0,0,0,0.8,80.92,-2.8,0.0,0.0,0.0,2.1,52.83,-4.4,0.0,0.0,0.0,2.4,57.25,-6.0,0.0,0.0,0.0,1.1,59.09,0.1,0.0,0.0,0.0,1.3,40.27,3.1,0.0,0.0,0.0,1.7,26.93,3.6,0.0
289,0.5,0,0,0.9,81.53,-2.9,0.0,0.0,0.0,2.0,54.44,-4.8,0.0,0.0,0.0,2.4,57.25,-6.0,0.0,0.0,0.0,1.2,61.20,-0.4,0.0,0.0,0.0,1.3,40.55,3.0,0.0,0.0,0.0,1.7,27.12,3.5,0.0
290,1.0,0,0,1.0,79.91,-3.0,0.0,0.0,0.0,1.9,52.78,-5.1,0.0,0.0,0.0,2.5,55.26,-6.1,0.0,0.0,0.0,1.3,65.76,-0.8,0.0,0.0,0.0,1.2,40.27,2.9,0.0,0.0,0.0,1.7,28.00,3.4,0.0
291,1.5,0,0,0.9,79.91,-3.0,0.0,0.0,0.0,1.8,53.59,-5.3,0.0,0.0,0.0,2.5,55.26,-6.1,0.0,0.0,0.0,1.4,66.24,-0.9,0.0,0.0,0.0,1.2,40.56,2.8,0.0,0.0,0.0,1.7,28.40,3.2,0.0
292,2.0,0,0,0.9,77.20,-3.0,0.0,0.0,0.0,1.7,52.63,-5.5,0.0,0.0,0.0,2.6,54.25,-6.0,0.0,0.0,0.0,1.5,69.70,-1.0,0.0,0.0,0.0,1.2,41.81,2.6,0.0,0.0,0.0,1.7,30.53,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,21.5,0,0,0.8,63.35,13.7,0.0,0.0,0.0,0.5,74.13,12.5,0.0,0.0,0.0,1.0,61.43,13.1,0.0,0.0,0.0,1.1,79.32,12.2,0.0,0.0,0.0,0.8,64.85,11.1,0.0,0.0,0.0,1.2,68.24,9.3,0.0
332,22.0,0,0,0.7,64.82,13.1,0.0,0.0,0.0,0.7,73.54,12.0,0.0,0.0,0.0,1.0,62.61,12.5,0.0,0.0,0.0,1.2,80.41,11.7,0.0,0.0,0.0,1.0,66.71,10.3,0.0,0.0,0.0,1.2,69.78,8.7,0.0
333,22.5,0,0,0.7,66.10,12.8,0.0,0.0,0.0,0.9,75.01,11.7,0.0,0.0,0.0,1.0,64.77,12.0,0.0,0.0,0.0,1.1,83.10,11.2,0.0,0.0,0.0,1.2,68.97,9.8,0.0,0.0,0.0,1.3,71.70,8.3,0.0
334,23.0,0,0,0.6,67.64,12.4,0.0,0.0,0.0,1.1,74.47,11.3,0.0,0.0,0.0,1.0,65.31,11.5,0.0,0.0,0.0,1.1,84.37,10.8,0.0,0.0,0.0,1.3,69.70,9.3,0.0,0.0,0.0,1.4,71.52,7.9,0.0


# kfold Train & Predict

In [15]:
# kfold를 위한 모델 정의

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', # quantile로 받아줘야 quatile regression이 가능합니다.
                          alpha=q, # 이건 나중에 quantile 받을 인자입니다.
                          n_estimators=100000, # 10000만번을 돌리겠다고? 미쳤어?
                          bagging_fraction=0.7,
                          learning_rate=0.027,
                          subsample=0.7)                   
                              
    model.fit(X_train,
              Y_train,
              eval_metric = ['quantile'],
              eval_set=[(X_valid, Y_valid)],
              early_stopping_rounds=300, # 300번 동안 loss가 줄지 않으면 멈춥니다.
              verbose=1000) # 1000번 마다 결과를 출력해줍니다.
    
    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [16]:
def oof_train_data(X, y, X_test, N_SPLIT=5):
    # kfold 셋으로 나눠서 교차 예측을 하고, 해당 결과를 앙상블해서 출력해주는 함수입니다.
    
    # 전처리 단계에서 index가 깨졌으니, 리셋해줍니다. 그래야 kfold로 잘 나눠집니다.
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # split을 기준으로 kfold instance를 받아줍니다.
    N_SPLIT = N_SPLIT
    kf = KFold(n_splits=N_SPLIT, shuffle=False, random_state=42)
    
    # 퀀틸을 미리 셋팅해줍니다.
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 먼저 pred 값을 받을 스키마를 만들어줍니다.
    oof_pred = pd.DataFrame(np.zeros((X_test.shape[0], len(quantiles))), columns=[0,0,0,0,0,0,0,0,0])
    oof_models = []
    
    # fold 별로 train, val을 찢고, 학습하고 예측까지 합니다.
    for i, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
        print("Kfold iteration count: ", i)
        print("------------------------------------------------")
        x_train, y_train = X.iloc[trn_idx, :], y[trn_idx]
        x_valid, y_valid = X.iloc[val_idx, :], y[val_idx]
        
        # 결과값을 받을 프레임을 잡아줍니다.
        LGBM_models = []
        LGBM_actual_pred = pd.DataFrame()

        # 퀀틸별로 모델 학습을 합니다.
        for q in quantiles:
            pred, model = LGBM(q, x_train, y_train, x_valid, y_valid, X_test)
            LGBM_models.append(model)
            LGBM_actual_pred = pd.concat([LGBM_actual_pred, pred],axis=1) #pred를 컨캣해줍니다.

        # ------ 여기까지가 한개의 폴드에 대한 pred 셋이 추출됩니다.
        # 전체 pred 스키마에 fold 별로 출력된 pred를 더해줍니다.(values 만 있기 때문에 값이 더해집니다.)
        oof_pred += LGBM_actual_pred
        # 나중에 쓸 일이 있으니 모델도 받아줍니다.
        oof_models += LGBM_models
        
    # values가 k개의 예측값이 더해졌으니, N_SPLIT의 수로 나눠줍니다. 평균을 내준다고 보면 됩니다.
    oof_pred = oof_pred/N_SPLIT    
    oof_pred.columns=quantiles # 컬럼은 퀀틸로 박아줍니다.
    
    return oof_models, oof_pred

In [105]:
%%time
# Target1
models_1, results_1 = oof_train_data(X, y_1, X_test, 5)

Kfold iteration count:  0
------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[83]	valid_0's quantile: 1.55726
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[98]	valid_0's quantile: 2.60735
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[150]	valid_0's quantile: 3.20743
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Earl

In [106]:
%%time
# Target2
models_2, results_2 = oof_train_data(X, y_2, X_test, 5)

Kfold iteration count:  0
------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[77]	valid_0's quantile: 1.58091
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[87]	valid_0's quantile: 2.71846
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[111]	valid_0's quantile: 3.4005
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
Early

In [108]:
# lag model 3 window decross model

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission[16:33]

#submission.to_csv('./submission/submission_210108-2_6days-oof-notshuffle.csv', index=False)

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
16,0.csv_Day7_8h00m,2.016,3.060,3.118,3.714,4.552,5.740,6.326,6.932,8.252
17,0.csv_Day7_8h30m,4.004,6.516,7.600,8.580,9.716,11.864,13.368,12.836,15.146
18,0.csv_Day7_9h00m,4.954,9.076,11.674,13.562,16.682,18.748,22.000,20.958,20.946
19,0.csv_Day7_9h30m,7.520,12.144,15.400,19.160,23.976,23.664,27.302,27.894,31.954
20,0.csv_Day7_10h00m,9.482,15.128,20.086,24.324,28.798,28.596,33.464,34.712,35.100
21,0.csv_Day7_10h30m,10.966,17.976,23.598,27.478,31.840,33.600,37.022,39.774,38.302
22,0.csv_Day7_11h00m,13.298,20.528,26.956,31.518,38.258,39.342,41.364,44.330,43.272
23,0.csv_Day7_11h30m,13.648,19.304,25.924,29.780,36.768,38.124,40.634,45.782,44.694
24,0.csv_Day7_12h00m,14.460,20.922,28.698,32.800,39.412,40.632,43.924,46.232,45.600
25,0.csv_Day7_12h30m,13.938,21.368,29.682,34.870,41.680,45.098,45.386,47.146,45.542


시각화는 보지 않겠습니다.

- oof_shuffle 모델은 1.938 정도, 학습시간도 아주 깁니다.
- oof_noshuffle 모델은 1.893 정도, 학습시간은 짧습니다.

현재 최고점이 1.890 수준입니다. 아쉽습니다. oof 앙상블로도 이러한 결과를 내는 것을 보면 기존 모델들이 과적합이 되지 않고 아주 일반화가 잘 된 모델이라는 것을 설명해줍니다.

즉, 해당 모델링이 나쁘지 않으니, 변수 생성에 심혈을 기울여야 성능이 올라갈 것입니다.

In [84]:
# Get the model and the predictions in (a) - (b)
def CATB(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = CatBoostRegressor(objective='Quantile',
                              #alpha=q,# quantile로 받아줘야 quatile regression이 가능합니다.
                              loss_function='Quantile:alpha={}'.format(q), # 이건 나중에 quantile 받을 인자입니다.
                              iterations=100000, # 10000만번을 돌리겠다고? 미쳤어?
                              learning_rate=0.027)#,
                              #task_type="GPU",
                              #devices='0:1')                   
                         
                         
    model.fit(X_train,
              Y_train,
              #eval_metric = ['quantile'],
              eval_set=[(X_valid, Y_valid)],
              early_stopping_rounds=300,
              verbose=500) # 대신 얼리 스토핑을 넣었네요.

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

# Target 예측

def train_catb(X_train, Y_train, X_valid, Y_valid, X_test):

    CATB_models=[]
    CATB_actual_pred = pd.DataFrame()

    for q in quantiles: # 9번 반복할 겁니다. quantile 마다.
        print(q)
        pred , model = CATB(q, # 퀀틸 박아줍니다.
                            X_train, Y_train, X_valid, Y_valid, X_test)
        CATB_models.append(model) # 모델을 어펜드 해줍니다.
        CATB_actual_pred = pd.concat([CATB_actual_pred, pred*(q+0.5)],axis=1) #pred를 컨캣해줍니다.

    CATB_actual_pred.columns=quantiles # 컬럼은 퀀틸로 박아줍니다.
    
    return CATB_models, CATB_actual_pred

In [74]:
tra = train[:35040]
va = train[35040:]

In [75]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [76]:
def make_train_test_val(X, val, features):

    x_train = X[features]
    y_train_1 = X['TARGET_1']
    y_train_2 = X['TARGET_2']

    x_val = val[features]
    y_val_1 = val['TARGET_1']
    y_val_2 = val['TARGET_2']
    
    print(x_train.shape)
    
    return x_train, y_train_1, y_train_2, x_val, y_val_1, y_val_2

In [82]:
x_train, y_train_1, y_train_2, x_val, y_val_1, y_val_2 = make_train_test_val(tra, va, tra.columns[:-2])

(35040, 37)


In [85]:
# Target1
models_1, results_1 = train_catb(x_train, y_train_1, x_val, y_val_1, X_test)
results_1.sort_index()[:48]

0.1
0:	learn: 8.6943800	test: 8.7318503	best: 8.7318503 (0)	total: 13.7ms	remaining: 22m 53s
500:	learn: 2.3143310	test: 2.7325839	best: 2.7260088 (320)	total: 3.67s	remaining: 12m 10s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.726008849
bestIteration = 320

Shrink model to first 321 iterations.
0.2
0:	learn: 8.6943800	test: 8.7318503	best: 8.7318503 (0)	total: 11.2ms	remaining: 18m 42s
500:	learn: 2.3143310	test: 2.7325839	best: 2.7260088 (320)	total: 3.77s	remaining: 12m 29s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.726008849
bestIteration = 320

Shrink model to first 321 iterations.
0.3
0:	learn: 8.6943800	test: 8.7318503	best: 8.7318503 (0)	total: 11.2ms	remaining: 18m 35s
500:	learn: 2.3143310	test: 2.7325839	best: 2.7260088 (320)	total: 3.78s	remaining: 12m 30s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.726008849
bestIteration = 320

Shrink model to first 321 iterations.
0.4
0:	learn: 8.6943800	test: 8

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
1,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
6,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
7,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
8,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
9,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000


In [86]:
# Target1
models_2, results_2 = train_catb(x_train, y_train_2, x_val, y_val_2, X_test)
results_2.sort_index()[:48]

0.1
0:	learn: 8.6979133	test: 8.7301330	best: 8.7301330 (0)	total: 11.7ms	remaining: 19m 27s
500:	learn: 2.3807856	test: 2.8233522	best: 2.8214752 (420)	total: 3.74s	remaining: 12m 23s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.821475183
bestIteration = 420

Shrink model to first 421 iterations.
0.2
0:	learn: 8.6979133	test: 8.7301330	best: 8.7301330 (0)	total: 13.2ms	remaining: 21m 58s
500:	learn: 2.3807856	test: 2.8233522	best: 2.8214752 (420)	total: 3.79s	remaining: 12m 32s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.821475183
bestIteration = 420

Shrink model to first 421 iterations.
0.3
0:	learn: 8.6979133	test: 8.7301330	best: 8.7301330 (0)	total: 11ms	remaining: 18m 24s
500:	learn: 2.3807856	test: 2.8233522	best: 2.8214752 (420)	total: 3.72s	remaining: 12m 19s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 2.821475183
bestIteration = 420

Shrink model to first 421 iterations.
0.4
0:	learn: 8.6979133	test: 8.7

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,-0.006,-0.007,-0.008,-0.009,-0.01,-0.011,-0.012,-0.013,-0.014
1,-0.006,-0.007,-0.008,-0.009,-0.01,-0.011,-0.012,-0.013,-0.014
2,-0.006,-0.007,-0.008,-0.009,-0.01,-0.011,-0.012,-0.013,-0.014
3,-0.006,-0.007,-0.008,-0.009,-0.01,-0.011,-0.012,-0.013,-0.014
4,-0.006,-0.007,-0.008,-0.009,-0.01,-0.011,-0.012,-0.013,-0.014
5,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
6,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
7,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
8,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
9,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000


In [87]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
#submission.to_csv('./submission/submission_210110-1_catb-test.csv', index=False)

In [171]:
def compute_quantile_loss(y_true, y_pred, quantile):
    residual = y_true - y_pred
    return np.maximum(quantile * residual, (quantile - 1) * residual)

In [172]:
# Get the model and the predictions in (a) - (b)
def XGB(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling
    params = {'objective':'reg:squarederror',
              'n_estimators':100000,
              'alpha':q,# quantile로 받아줘야 quatile regression이 가능합니다.
              'learning_rate':0.027,
              'subsample':0.7,
              'n_jobs':-1}
    
    model = XGBRegressor(**params)                  
                         
                         
    model.fit(X_train,
              Y_train,
              eval_set=[(X_valid, Y_valid)],
              early_stopping_rounds=300,
              verbose=500) # 대신 얼리 스토핑을 넣었네요.

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

# Target 예측

def train_xgb(X_train, Y_train, X_valid, Y_valid, X_test):

    XGB_models=[]
    XGB_actual_pred = pd.DataFrame()

    for q in quantiles: # 9번 반복할 겁니다. quantile 마다.
        print(q)
        pred , model = XGB(q, # 퀀틸 박아줍니다.
                            X_train, Y_train, X_valid, Y_valid, X_test)
        XGB_models.append(model) # 모델을 어펜드 해줍니다.
        XGB_actual_pred = pd.concat([XGB_actual_pred, pred*(q+0.5)],axis=1) #pred를 컨캣해줍니다.

    XGB_actual_pred.columns=quantiles # 컬럼은 퀀틸로 박아줍니다.
    
    return XGB_models, XGB_actual_pred

In [173]:
# Target1
models_1, results_1 = train_xgb(x_train, y_train_1, x_val, y_val_1, X_test)
results_1.sort_index()[:48]

0.1
[0]	validation_0-rmse:30.47205
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[177]	validation_0-rmse:11.68234

0.2
[0]	validation_0-rmse:30.47489
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[166]	validation_0-rmse:11.67502

0.3
[0]	validation_0-rmse:30.47490
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[167]	validation_0-rmse:11.66745

0.4
[0]	validation_0-rmse:30.47491
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[171]	validation_0-rmse:11.66998

0.5
[0]	validation_0-rmse:30.47491
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[178]	validation_0-rmse:11.65861

0.6
[0]	validation_0-rmse:30.47493
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[178]	validation_0-rmse:11.67080

0.7
[0]	validation_0-rmse:30.47494

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.054000,0.063000,0.064000,0.081000,0.160000,0.176000,0.096000,0.117000,0.168000
1,0.054000,0.063000,0.064000,0.081000,0.160000,0.176000,0.096000,0.117000,0.168000
2,0.054000,0.063000,0.064000,0.081000,0.160000,0.176000,0.096000,0.117000,0.168000
3,0.054000,0.063000,0.064000,0.081000,0.160000,0.176000,0.096000,0.117000,0.168000
4,0.060000,0.077000,0.080000,0.090000,0.090000,0.121000,0.120000,0.130000,0.140000
5,0.060000,0.077000,0.080000,0.090000,0.090000,0.121000,0.120000,0.130000,0.140000
6,0.060000,0.077000,0.080000,0.090000,0.100000,0.121000,0.120000,0.130000,0.140000
7,0.060000,0.077000,0.080000,0.090000,0.100000,0.121000,0.120000,0.130000,0.140000
8,0.060000,0.077000,0.080000,0.090000,0.100000,0.121000,0.120000,0.130000,0.140000
9,0.060000,0.077000,0.080000,0.090000,0.100000,0.121000,0.120000,0.130000,0.140000


In [174]:
# Target1
models_2, results_2 = train_xgb(x_train, y_train_2, x_val, y_val_2, X_test)
results_2.sort_index()[:48]

0.1
[0]	validation_0-rmse:30.48043
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[153]	validation_0-rmse:11.88597

0.2
[0]	validation_0-rmse:30.48044
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[157]	validation_0-rmse:11.88263

0.3
[0]	validation_0-rmse:30.48045
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[157]	validation_0-rmse:11.89025

0.4
[0]	validation_0-rmse:30.48046
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[157]	validation_0-rmse:11.87707

0.5
[0]	validation_0-rmse:30.48047
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[146]	validation_0-rmse:11.87863

0.6
[0]	validation_0-rmse:30.48048
Will train until validation_0-rmse hasn't improved in 300 rounds.
Stopping. Best iteration:
[157]	validation_0-rmse:11.87486

0.7
[0]	validation_0-rmse:30.48049

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
1,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
2,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
3,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
4,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
5,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
6,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
7,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
8,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000
9,0.048000,0.056000,0.072000,0.081000,0.080000,0.099000,0.108000,0.117000,0.126000


In [175]:
# xgb result에서 0이 아닌 0~0.1 사이의 값이 너무 많습니다.
# score에 감점이 될까하여 0으로 전부 맞춰주겠습니다.

for r in range(results_1.shape[0]):
    for c in range(results_1.shape[1]):
        if results_1.values[r, c] < 0.1:
            results_1.values[r, c] = 0
            
for r in range(results_2.shape[0]):
    for c in range(results_2.shape[1]):
        if results_2.values[r, c] < 0.1:
            results_2.values[r, c] = 0
    
# 0.01 좋아집니다. 핵심을 못 맞추는 걸로...

In [179]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission[14:33]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
14,0.csv_Day7_7h00m,0.000000,0.000000,0.000000,0.000000,0.100000,0.110000,0.120000,0.156000,0.140000
15,0.csv_Day7_7h30m,0.708000,0.756000,0.880000,0.999000,1.120000,1.188000,1.344000,1.482000,1.540000
16,0.csv_Day7_8h00m,2.592000,2.926000,3.320000,3.780000,4.250000,4.576000,5.028000,5.447000,5.754000
17,0.csv_Day7_8h30m,3.744000,4.473000,5.112000,5.634000,6.420000,6.820000,7.272000,8.436999,8.848001
18,0.csv_Day7_9h00m,8.526000,9.828000,11.072001,12.528000,14.070000,14.916000,16.320002,18.940998,20.034000
19,0.csv_Day7_9h30m,10.956000,13.027000,14.600000,15.912000,18.280001,20.460001,22.020000,24.543999,25.942001
20,0.csv_Day7_10h00m,11.838000,14.280000,16.184000,17.775000,20.469999,22.583002,23.351999,26.103998,28.350000
21,0.csv_Day7_10h30m,15.558001,16.982000,19.496000,22.032000,24.620001,27.082001,30.240002,33.279999,36.483997
22,0.csv_Day7_11h00m,18.252001,19.957001,22.303999,25.766998,28.530001,32.307003,34.560001,40.897999,43.189999
23,0.csv_Day7_11h30m,16.746000,19.145000,21.176001,24.074999,27.110001,30.316000,33.384003,37.388000,39.438000


In [178]:
#submission.to_csv('./submission/submission_210110-4_xgb-test-setparam.csv', index=False)

set param 버전이 2.2에서 2.1로 올랐습니다. 꽤 높게 올려주는군요! 그러나 아직 멀었습니다.